## Naive Bayes and Vector Embeddings

### Importing the data and libraries

In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from collections import defaultdict

In [2]:
# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Split into training and test sets
X_train, X_test, y_train, y_test = newsgroups.data[:1000], newsgroups.data[1000:], newsgroups.target[:1000], newsgroups.target[1000:]

# Categories for evaluation
categories = newsgroups.target_names

print(f"Categories: {categories}")

Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Preprocessing

In [3]:
import string
import re
def preprocess_text(text):
    """Convert text to lowercase and remove punctuation and non alpha numeric characters using regex"""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    return text

# Apply preprocessing to training data
X_train = [preprocess_text(text) for text in X_train]
X_test = [preprocess_text(text) for text in X_test]


### Feature Extraction

Here create 2 functions: Bag of Words and TF-IDF

##### **Bag of Words (BoW)**

First create the vocabulary for the bag of words, in layman terms get the set of all unique words

In [4]:
def create_vocabulary(text_data):
    """Create a vocabulary from the provided text data."""
    vocabulary = set()
    for text in text_data:
        words = text.split()
        vocabulary.update(words)
    return list(vocabulary)

# Create vocabulary from the training data
vocabulary = create_vocabulary(X_train)

Next create a function to compute the number of occurences of that word given the vocabulary and text, return a dictionary

In [5]:
def BoW(text, vocabulary):
    """Convert a text document into a vector based on the vocabulary."""
    word_count = {word: 0 for word in vocabulary}
    words = text.split()
    for word in words:
        if word in word_count:
            word_count[word] += 1
    return list(word_count.values())

#### **TF-IDF**

##### Theory

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a numerical statistic used to evaluate the importance of a word within a document relative to a collection (or corpus) of documents. It is commonly used in text mining and information retrieval tasks such as text classification, clustering, and document retrieval.

##### 1. **Term Frequency (TF)**

Term Frequency measures how frequently a term (word) appears in a document. It is calculated as:

$$
\text{TF}(t, d) = \frac{\text{Number of times term t appears in document d}}{\text{Total number of terms in document d}}
$$

##### 2. **Inverse Document Frequency (IDF)**

Inverse Document Frequency measures how important a term is in the entire corpus. It reduces the weight of common words that appear in many documents. The IDF is calculated as:

$$
\text{IDF}(t) = \log \frac{N}{df(t)}
$$

- Where:
  - $ N $ is the total number of documents in the corpus.
  - $ df(t) $ is the number of documents that contain the term \$$t \).

##### 3. **TF-IDF Score**

The **TF-IDF score** is the product of the Term Frequency (TF) and Inverse Document Frequency (IDF):

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)
$$

- This score reflects the importance of a term in a document relative to its rarity across the entire corpus.


##### 4. **Smoothed IDF (Inverse Document Frequency)**

The formula for **IDF** can be smoothed by adding a constant to the document frequency count in the denominator. This avoids division by zero and prevents excessively high values for terms that appear in most documents.

**Smoothed IDF Formula:**

$$
\text{IDF}_{\text{smooth}}(t) = \log \left( \frac{N + 1}{df(t) + 1} \right) + 1
$$

Where:
- $ N $ = Total number of documents in the corpus.
- $ df(t) $ = Number of documents that contain the term $ t $.
- The `+1` in both the numerator and denominator ensures that the term frequency is never zero.

##### 5. **Smoothed TF (Term Frequency)**

A common smoothing technique for **Term Frequency** is to apply **logarithmic scaling**. Instead of using the raw term count, the logarithm of the term frequency is used to downscale the impact of very frequent terms.

**Smoothed TF Formula:**

$$
\text{TF}_{\text{smooth}}(t, d) = 1 + \log(\text{count}(t, d))
$$

Where:
- If the count of term $t$ in document $d$ is 0, the formula results in 1 (to avoid zero counts).

##### 6. **Complete Smoothed TF-IDF**

The final **TF-IDF score with smoothing** combines both smoothed **TF** and **IDF** formulas:

$$
\text{TF-IDF}_{\text{smooth}}(t, d) = \left( 1 + \log(\text{count}(t, d)) \right) \times \log \left( \frac{N + 1}{df(t) + 1} \right) + 1
$$

This formula:
- Applies logarithmic smoothing to **TF**.
- Applies smoothing to **IDF** to avoid division by zero and prevent extreme values for very common words.


##### Code

We will be using the smoothned version of TF-IDF, first create the function for TF

In [6]:
from collections import Counter

# Function to compute Term Frequency (TF) with smoothing
def term_frequency(document, smoothing=True):
    """
    Compute the Term Frequency (TF) for each term in a document with optional smoothing.
    
    :param document: List of words in a document.
    :param smoothing: Boolean flag for applying Laplace smoothing.
    :return: Dictionary with term frequency for each term.
    """
    term_count = Counter(document)
    total_terms = len(document)
    
    tf = {}
    
    # Count the frequency of each term in the document
    for term, count in term_count.items():
    # Apply Laplace smoothing if specified
        if smoothing:
            tf[term] = (count + 1) / (total_terms + len(term_count))
        else:
            tf[term] = count / total_terms
    
    return tf

Now create a function for IDF

In [7]:
import math

# Function to compute Inverse Document Frequency (IDF)
def inverse_document_frequency(documents):
    """
    Compute the Inverse Document Frequency (IDF) for each term in the corpus.
    
    :param documents: List of documents (each document is a list of words).
    :return: Dictionary with inverse document frequency for each term.
    """
    total_documents = len(documents)
    document_frequency = {}

    # Count how many documents each term appears in
    for document in documents:
        unique_terms = set(document)
        for term in unique_terms:
            document_frequency[term] = document_frequency.get(term, 0) + 1

    # Calculate IDF for each term
    idf = {}
    for term, count in document_frequency.items():
        idf[term] = math.log(total_documents / (1 + count))

    
    return idf

Combining

In [8]:
# Function to compute TF-IDF for each term in each document
def tfidf_features(documents, smoothing=True):
    """
    Compute the TF-IDF for each term in each document with optional smoothing on TF.
    
    :param documents: List of documents (each document is a list of words).
    :param smoothing: Boolean flag for applying Laplace smoothing to TF.
    :return: List of TF-IDF vectors for each document.
    """
    # Step 1: Compute IDF for the corpus
    idf = inverse_document_frequency(documents)
    tfidf_vectors = []
    
    # Step 2: Compute TF for each document and then compute TF-IDF
    for document in documents:
        tf = term_frequency(document, smoothing)
        tfidf_vector = {}
        
        for term in document:
            tfidf_vector[term] = tf[term] * idf.get(term, 0)

        tfidf_vectors.append(tfidf_vector)
    
    return tfidf_vectors, list(idf.keys())

#### Implementing Naive Bayes

In [9]:
from collections import defaultdict

class NaiveBayes:
    def __init__(self, method='bow'):
        self.method = method
        self.class_probs = {}
        self.feature_probs = defaultdict(lambda: defaultdict(float))

    def fit(self, X, y):
        class_counts = defaultdict(int)
        for label in y:
            class_counts[label] += 1

        total_documents = len(y)
        for label, count in class_counts.items():
            self.class_probs[label] = count / total_documents

        feature_counts = defaultdict(lambda: defaultdict(int))
        for i, document in enumerate(X):
            label = y[i]
            for word in document:
                feature_counts[label][word] += 1

        for label, word_counts in feature_counts.items():
            total_words_in_class = sum(word_counts.values()) + len(word_counts)
            for word, count in word_counts.items():
                self.feature_probs[label][word] = (count + 1) / total_words_in_class

    import numpy as np

    def predict(self, X):
        predictions = []
        for document in X:
            class_scores = {}
            for label in self.class_probs:
                score = np.log(self.class_probs[label])
                if self.method == 'bow':
                    for word in document:
                        score += np.log(self.feature_probs[label].get(word, 1e-5))
                elif self.method == 'tfidf':
                    if isinstance(document, dict):  # Check if it's a dictionary for TF-IDF
                        for word, tfidf_value in document.items():
                            score += np.log(tfidf_value * self.feature_probs[label].get(word, 1e-5) if tfidf_value > 0 else 1e-5)
                class_scores[label] = score

            predicted_class = max(class_scores, key=class_scores.get)
            predictions.append(predicted_class)

        return np.array(predictions)


In [11]:
# Example usage for TF-IDF
nb = NaiveBayes(method='tfidf')
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)
print() 

[5 5 5 ... 5 5 5]


#### Accuracy for BoW and TFDIF

In [ ]:
import numpy as np

def accuracy(y_true, y_pred):
    correct = np.sum(y_true == y_pred)
    return correct / len(y_true)

nb_bow = NaiveBayes(method='bow')
nb_bow.fit(X_train, y_train)
predictions_bow = nb_bow.predict(X_test)
accuracy_bow = accuracy(y_test, predictions_bow)
print(f"Accuracy (BoW): {accuracy_bow * 100:.2f}%")

nb_tfidf = NaiveBayes(method='tfidf')
nb_tfidf.fit(X_train, y_train)
predictions_tfidf = nb_tfidf.predict(X_test)
accuracy_tfidf = accuracy(y_test, predictions_tfidf)
print(f"Accuracy (TF-IDF): {accuracy_tfidf * 100:.2f}%")
